# Prepare Amenities Features
In this notebook we prepare amenities features for our network.  Amenities include features such as restaurants, cafes, bars, and schools.

In [ ]:
%matplotlib inline

import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np
import geopandas  as gpd
import pandas as pd
ox.config(use_cache=True, log_console=False)
%matplotlib inline
import h3 as h3

import boto3
import awswrangler
# set name of S3 bucket
s3_bucket = 'traffic-data-bucket'

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

import os
from os.path import isfile, join
from pathlib import Path
from os import listdir
import os
os.getcwd()

path =  Path(os.getcwd())
root = path.parent.absolute()

h3_level = 8
root

## 1. Create Boto3 session
Start by creating a boto3 session so that we can connect to the S3 bucket.

In [ ]:
from aws_secrets import aws_access_key_id, aws_secret_access_key, aws_session_token

my_session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_session_token = aws_session_token

)

## 2. Extract amenities for Los Angeles County

In [ ]:
#G = ox.graph_from_place('Los Angeles County, CA, USA', network_type='drive')
G = ox.geometries.geometries_from_place('Los Angeles County, CA, USA', tags = {'amenity':True, 'landuse':['retail','commercial']}, which_result=None, buffer_dist=None)
G.head()

In [ ]:
G.amenity.value_counts().head(75)

### 2.1 Get features for restaurants and cafes

In [ ]:
filt_G = G[G.amenity.isin(['restaurant', 'cafe'])]
filt_G = filt_G.to_crs(epsg=4326)
centroids = filt_G.centroid
lon = pd.Series(centroids.x).reset_index(drop = True)
lat = pd.Series(centroids.y).reset_index(drop = True)

rest_df = pd.concat([lon, lat], axis=1)
rest_df.columns = ['lon', 'lat']
rest_df.sample(2)


### 2.2 Get features for schools

In [ ]:
filt_G = G[G.amenity.isin(['school','kindergarten'])]
filt_G = filt_G.to_crs(epsg=4326)
centroids = filt_G.centroid
lon = pd.Series(centroids.x).reset_index(drop = True)
lat = pd.Series(centroids.y).reset_index(drop = True)

school_df = pd.concat([lon, lat], axis=1)
school_df.columns = ['lon', 'lat']
school_df.sample(2)


### 2.3 Get features for bars

In [ ]:
filt_G = G[G.amenity.isin(['bar', 'pub'])]
filt_G = filt_G.to_crs(epsg=4326)
centroids = filt_G.centroid
lon = pd.Series(centroids.x).reset_index(drop = True)
lat = pd.Series(centroids.y).reset_index(drop = True)

bar_df = pd.concat([lon, lat], axis=1)
bar_df.columns = ['lon', 'lat']
bar_df.sample(2)


### 2.4 Get features for universities and colleges

In [ ]:
filt_G = G[G.amenity.isin(['university', 'college'])]
filt_G = filt_G.to_crs(epsg=4326)
centroids = filt_G.centroid
lon = pd.Series(centroids.x).reset_index(drop = True)
lat = pd.Series(centroids.y).reset_index(drop = True)

college_df = pd.concat([lon, lat], axis=1)
college_df.columns = ['lon', 'lat']
college_df.sample(2)


In [ ]:
#import contextily as ctx
#base = centroids.plot(marker='o', color='red', markersize=2)

#ctx.add_basemap(base)
#plt.show()

## 3. Create a dataframe with the count of each feature for per hexagon

In [ ]:
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row.lat, row.lon, h3_level)


rest_df['hex_id'] = rest_df.apply(lat_lng_to_h3, axis=1)
school_df['hex_id'] = school_df.apply(lat_lng_to_h3, axis=1)
bar_df['hex_id'] = bar_df.apply(lat_lng_to_h3, axis=1)
college_df['hex_id'] = college_df.apply(lat_lng_to_h3, axis=1)

In [ ]:
rest_hex_cnt_df = rest_df.groupby(['hex_id']).hex_id.agg('count').to_frame('count').reset_index()
school_hex_cnt_df = school_df.groupby(['hex_id']).hex_id.agg('count').to_frame('count').reset_index()
bar_hex_cnt_df = bar_df.groupby(['hex_id']).hex_id.agg('count').to_frame('count').reset_index()
college_hex_cnt_df = college_df.groupby(['hex_id']).hex_id.agg('count').to_frame('count').reset_index()
rest_hex_cnt_df.sample(3)

In [ ]:
rest_hex_cnt_df.columns = ['hex_id', 'amenities_restaurant_cnt']
school_hex_cnt_df.columns = ['hex_id', 'amenities_school_cnt']
bar_hex_cnt_df.columns = ['hex_id', 'amenities_bar_cnt']
college_hex_cnt_df.columns = ['hex_id', 'amenities_college_cnt']

In [ ]:
ammeneties_hex_cnt_df = rest_hex_cnt_df.merge(school_hex_cnt_df, how = 'outer', on = 'hex_id')
ammeneties_hex_cnt_df = ammeneties_hex_cnt_df.merge(bar_hex_cnt_df, how = 'outer', on = 'hex_id')
ammeneties_hex_cnt_df = ammeneties_hex_cnt_df.merge(college_hex_cnt_df, how = 'outer', on = 'hex_id')
ammeneties_hex_cnt_df

## 4. Upload to S3 bucket

In [ ]:
awswrangler.s3.to_csv(df=ammeneties_hex_cnt_df, path = f's3://{s3_bucket}/nodes_and_edges/la_county_amenities/la_county_ammenities.csv', index=False,
                       boto3_session=my_session, use_threads=True
                       )

## 5. Store Local Copy of Sample Data

In [ ]:
ammeneties_hex_cnt_df[0:50].to_csv(root / 'X.data' / 'nodes_and_edges' / 'la_county_amenities' / 'la_county_ammenities.csv', index = False )